In [1]:
import notebook_common
import jaconv
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    r_word_2400_xlsx,
    w_word_2400_csv,
    w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    pattern_table_written,
    pattern_td_written,
    r_written_freq_nlt_with_enter_txt,
    w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    # w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_tmp_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    l,
)

p("Import common module")

Import common module


In [2]:
with open(r_written_freq_nlt_with_enter_txt, "r", encoding="utf-8") as f:
    lst = []
    content = f.read()  # 一次性读取
    table_matches = pattern_table_written.findall(content)  # 或 finditer 迭代匹配
    for table_match in table_matches:
        td_matches_lst = pattern_td_written.findall(table_match)
        lst.append(td_matches_lst)
    columns = ["headword", "reading", "roman", "freq", "freq_per", "rank"]
    df = pd.DataFrame(lst, columns=columns)
    to_csv_sig(df, w_written_freq_nlt_with_enter_csv)
    p("w_written_freq_nlt_with_enter_csv--ok")
df

w_written_freq_nlt_with_enter_csv--ok


,headword,reading,roman,freq,freq_per,rank
0,110番,ヒャクトオバン,hyakutooban,108,0.0252,0.69889
1,110番する,ヒャクトオバンスル,hyakutoobansuru,7,0.0016,0.28913
2,1つ,ヒトツ,hitotsu,"4,366",1.0167,0.98121
3,1つする,ヒトツスル,hitotsusuru,2,0.0005,0.14991
4,2ラン,ツーラン,tsuran,99,0.0231,0.68579
...,...,...,...,...,...,...
85106,齲歯,ウバ,uba,19,0.0044,0.41736
85107,龍眼,リュウガン,ryuugan,13,0.0030,0.36012
85108,（有）,ユウゲンガイシャ,yuugengaisha,115,0.0268,0.70698
85109,（社）,シャダンホウジン,shadanhoujin,2,0.0005,0.09486


In [3]:
df.describe()

,headword,reading,roman,freq,freq_per,rank
count,85111,85111,85111,85111,85111,85107
unique,84286,66611,66096,4504,4504,85107
top,する,キコウ,kikou,1,0.0002,0.56826
freq,72,23,23,8074,8074,1


In [4]:
# df_ff_4 = pd.read_csv(
#     "D:/04.japandict/Forvo/ff-4.csv",
#     encoding="utf-8-sig",
#     on_bad_lines="skip",
#     engine="python",
# )
# df_ff_4.dropna(subset=["headword", "mp3_path"], inplace=True)

# df_ff_4["headword"] = df_ff_4["headword"].apply(jaconv.kata2hira)

# df_ff_5 = (
#     df_ff_4.groupby("headword")
#     .agg({"mp3_path": lambda x: "|".join(x[x != ""])})  # 字符串拼接  # 求平均值
#     .reset_index()
# )
# to_csv_sig(df_ff_5,"d:/tmp/dd_ff_5.csv")
# l()

In [5]:
# 指定列中，只要有一列的数据为空，删除该行,Inplace=True

# 如果headword中出现'-非自立'结尾，则删除
df['headword'] = df['headword'].str.replace(r'-非自立$', '', regex=True)

na_columns = ["headword", "reading", "freq", "rank"]
df.dropna(subset=na_columns, inplace=True)
# to_deleted_columns = ["rank", "freq_per", "roman"]
to_deleted_columns = ["rank", "freq_per"]
# 将无用的列删除
df.drop(to_deleted_columns, axis=1, inplace=True)
# 将词频列的字符串转化为数字
df["freq"] = pd.to_numeric(df["freq"].str.replace(",", ""), errors="coerce")
# 将发音reading转化为平假名
df["hira"] = df["reading"].apply(jaconv.kata2hira)
l("df.describe()")
p(df.describe())
l("df.dtypes")
p(df.dtypes)
l("df")
p(df)

----------------------------------------df.describe()----------------------------------------
                freq
count   85107.000000
mean      504.548862
std      6804.399540
min         1.000000
25%         6.000000
50%        32.000000
75%       153.000000
max    968064.000000
----------------------------------------df.dtypes----------------------------------------
headword    object
reading     object
roman       object
freq         int64
hira        object
dtype: object
----------------------------------------df----------------------------------------
      headword    reading            roman  freq       hira
0         110番    ヒャクトオバン      hyakutooban   108    ひゃくとおばん
1       110番する  ヒャクトオバンスル  hyakutoobansuru     7  ひゃくとおばんする
2           1つ        ヒトツ          hitotsu  4366        ひとつ
3         1つする      ヒトツスル      hitotsusuru     2      ひとつする
4          2ラン       ツーラン           tsuran    99       つーらん
...        ...        ...              ...   ...        ...
85106       齲歯 

In [6]:
df["count-head-read"] = df.groupby(["headword", "reading"]).transform("size")
df["count-head-read-roman"] = df.groupby(["headword", "reading","roman"]).transform("size")
df

,headword,reading,roman,freq,hira,count-head-read,count-head-read-roman
0,110番,ヒャクトオバン,hyakutooban,108,ひゃくとおばん,1,1
1,110番する,ヒャクトオバンスル,hyakutoobansuru,7,ひゃくとおばんする,1,1
2,1つ,ヒトツ,hitotsu,4366,ひとつ,1,1
3,1つする,ヒトツスル,hitotsusuru,2,ひとつする,1,1
4,2ラン,ツーラン,tsuran,99,つーらん,1,1
...,...,...,...,...,...,...,...
85106,齲歯,ウバ,uba,19,うば,1,1
85107,龍眼,リュウガン,ryuugan,13,りゅうがん,1,1
85108,（有）,ユウゲンガイシャ,yuugengaisha,115,ゆうげんがいしゃ,1,1
85109,（社）,シャダンホウジン,shadanhoujin,2,しゃだんほうじん,1,1


In [7]:
# 临时文件
to_csv_sig(df,'d:/tmp/written_freq_tmp1.csv')
p('written_freq_tmp1.csv--ok')

written_freq_tmp1.csv--ok


In [8]:
# 以拼写,发音,罗马音来分组，对词频进行汇总
summed_df = (
    df["freq"].groupby([df["headword"], df["hira"], df["roman"]]).sum().to_frame()
)
summed_df.rename(columns={"freq": "written_freq"}, inplace=True)
summed_df.sort_values(by="written_freq", ascending=False, inplace=True)
to_csv_sig(summed_df, w_written_freq_nlt_sumed_csv, need_index=True)
p("w_written_freq_nlt_sumed_csv--ok")
summed_df

w_written_freq_nlt_sumed_csv--ok


,,,written_freq
headword,hira,roman,
いる,いる,iru,1067608
こと,こと,koto,701470
の,の,no,686025
する,する,suru,607405
なる,なる,naru,528539
...,...,...,...
焼きいもする,やきいもする,yakiimosuru,1
へろへろする,へろへろする,heroherosuru,1
へなへなする,へなへなする,henahenasuru,1


In [9]:
summed_df.index

MultiIndex([(    'いる',      'いる',          'iru'),
            (    'こと',      'こと',         'koto'),
            (     'の',       'の',           'no'),
            (    'する',      'する',         'suru'),
            (    'なる',      'なる',         'naru'),
            (    'ある',      'ある',          'aru'),
            (    'その',      'その',         'sono'),
            (    'よう',      'よう',          'you'),
            (    '言う',      'いう',           'iu'),
            (    'この',      'この',         'kono'),
            ...
            ( '焦がれ死ぬ',   'こがれしぬ',  'kogareshinu'),
            (    '無芸',     'むげい',        'mugei'),
            (   '無芸な',    'むげいな',      'mugeina'),
            (  '焦りする',   'あせりする',    'aserisuru'),
            (  '焦燥する', 'しょうそうする',  'shousousuru'),
            ('焼きいもする',  'やきいもする',  'yakiimosuru'),
            ('へろへろする',  'へろへろする', 'heroherosuru'),
            ('へなへなする',  'へなへなする', 'henahenasuru'),
            (   'へな振',    'へなぶり',     'henaburi'),
            (  